# NanoEventsFactory example

Load a single file from x-cache (using redirector_ucsd) or the grid (using redirector_fnal), and play with the events.


In [24]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection

from Tools.config_helpers import redirector_fnal, redirector_ucsd
from Tools.nano_mapping import make_fileset
from Tools.helpers import get_samples

import awkward as ak

samples = get_samples(2018)

#fileset = make_fileset(['DY'], 2018, redirector=redirector_ucsd, small=True)

#fileset = {'/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM': ['022107FA-F567-1B44-B139-A18ADC996FCF.root']}

fileset = {'DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/70000/': ['root://xcache-redirector.t2.ucsd.edu:2040//store/mc/RunIIAutumn18NanoAODv7/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/70000/E074AFA3-6B0E-A74B-8D47-04C9A3D4DF4E.root']} 

# load a subset of events
n_max = 5000
events = NanoEventsFactory.from_root(
    fileset[list(fileset.keys())[0]][0],
    schemaclass = NanoAODSchema,
    entry_stop = n_max).events()



samples_2018.yaml


In [10]:
list(fileset.keys())[0]

'/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'

In [25]:
events.event

<Array [102441439, 102441465, ... 31313168] type='5000 * uint64[parameters={"__d...'>

In [13]:
ev = events[events.event==4590003]

In [14]:
ak.to_pandas(ev.MET)

,MetUnclustEnUpDeltaX,MetUnclustEnUpDeltaY,covXX,covXY,covYY,fiducialGenPhi,fiducialGenPt,phi,pt,significance,sumEt,sumPtUnclustered
entry,,,,,,,,,,,,


In [26]:
import time

from Tools.objects import Collections

start_time = time.time()
electron = Collections(events, 'Electron', 'tight', verbose=1).get()
muons = Collections(events, 'Muon', 'tight', verbose=1).get()


delta_time = time.time()-start_time

print ("\nTook %s seconds"%delta_time)


## Electron selection for WP tight ##
 - pt >= 15
 - abs(etaSC) <= 2.4
 - convVeto == 1
 - lostHits == 0
 - tightCharge == 2
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.1
 - abs(sip3d) <= 4.0
 - tight electron MVA ID
 - custom multi isolation
 - trigger safe
 - custom ID and multi-isolation
Found 1249 objects passing the selection

## Muon selection for WP tight ##
 - pt >= 15
 - abs(eta) <= 2.4
 - mediumId == 1
 - looseId == 1
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.05
 - abs(sip3d) <= 4
 - ptErrRel <= 0.2
 - custom multi isolation
 - custom multi-isolation
Found 1907 objects passing the selection

Took 1.3499305248260498 seconds


In [27]:
electron = electron[(electron.pt > 20) & (abs(electron.eta) < 2.4)]
electron = electron[( (electron.genPartIdx >= 0) & (abs(electron.matched_gen.pdgId)==11) )]

In [28]:
electrons.pt.tolist()

[[],
 [44.26520538330078, 43.29914855957031],
 [],
 [],
 [],
 [],
 [],
 [56.22705841064453],
 [],
 [],
 [],
 [],
 [29.118000030517578],
 [],
 [],
 [],
 [],
 [],
 [38.267398834228516, 33.25651550292969],
 [],
 [],
 [],
 [],
 [67.08100891113281],
 [],
 [],
 [],
 [],
 [],
 [35.821434020996094],
 [],
 [19.57379913330078],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [24.05095100402832],
 [],
 [48.7871208190918],
 [19.16155242919922],
 [],
 [],
 [],
 [],
 [],
 [],
 [21.160085678100586],
 [],
 [],
 [],
 [],
 [],
 [],
 [28.964889526367188],
 [],
 [],
 [],
 [],
 [31.97122573852539],
 [],
 [],
 [],
 [],
 [46.36690902709961],
 [27.629087448120117],
 [],
 [],
 [],
 [32.956275939941406],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [46.31135559082031],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [44.04412078857422, 40.605709075927734],
 [24.356609344482422],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

You can do anything that you're usually doing inside the processor here as well, so this is very useful for debugging

In [47]:
matched_electrons = electrons[electrons.genPartIdx>=0]
sum(ak.num(matched_electrons, axis=1))

1776

In [11]:
f_in = '/store/mc/RunIIAutumn18NanoAODv7/QCD_Pt-120to170_MuEnrichedPt5_TuneCP5_13TeV_pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/70000/DE335891-829A-B943-99BE-E5A179F5F3EB.root'

events = NanoEventsFactory.from_root(
    redirector_ucsd + f_in,
    schemaclass = NanoAODSchema,
    entry_stop = 9999999).events()

muons = Collections(events, 'Muon', 'tight', verbose=1).get()

single_mu_ev = events[ak.num(muons)>0]

event_list_tight = single_mu_ev.event
event_list_tight


## Muon selection for WP tight ##
 - pt >= 15
 - abs(eta) <= 2.4
 - mediumId == 1
 - looseId == 1
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.05
 - abs(sip3d) <= 4
 - ptErrRel <= 0.2
 - custom multi isolation
 - custom multi-isolation
Found 652 objects passing the selection


<Array [29552, 33557, ... 12024433, 12035747] type='651 * uint64[parameters={"__...'>

In [12]:
import numpy as np
event_selector = (np.zeros_like(events.MET.pt) == 1)

In [13]:
my_events = [29552, 12024433]

In [14]:
for ev in my_events:
    event_selector = (event_selector | (events.event == ev))

In [28]:
mu_df = ak.to_pandas(ak.flatten(muons[event_selector][fields_to_show]))
mu_df

,pt,conePt,eta,phi,miniPFRelIso_all,deepJet
entry,,,,,,
0,22.196507,73.3500,-0.518066,-0.862671,0.101121,0.974121
1,29.695017,93.2625,-1.611572,-1.919434,0.100588,0.124390


In [35]:
ev_df = ak.to_pandas(events.MET[event_selector])
ev_df

,MetUnclustEnUpDeltaX,MetUnclustEnUpDeltaY,covXX,covXY,covYY,fiducialGenPhi,fiducialGenPt,phi,pt,significance,sumEt,sumPtUnclustered
entry,,,,,,,,,,,,
0,-7.730469,-7.636719,860.0,-62.375000,1500.0,-1.039062,9.304688,0.413147,49.778206,2.777344,2454.0,1626.0
1,-5.054688,-2.496094,514.0,-5.484375,390.0,-1.850586,20.265625,-1.706299,52.056957,6.945312,1174.0,1016.0


In [36]:
import pandas as pd
pd.concat([mu_df, ev_df], axis=1, )

,pt,conePt,eta,phi,miniPFRelIso_all,deepJet,MetUnclustEnUpDeltaX,MetUnclustEnUpDeltaY,covXX,covXY,covYY,fiducialGenPhi,fiducialGenPt,phi,pt,significance,sumEt,sumPtUnclustered
entry,,,,,,,,,,,,,,,,,,
0,22.196507,73.3500,-0.518066,-0.862671,0.101121,0.974121,-7.730469,-7.636719,860.0,-62.375000,1500.0,-1.039062,9.304688,0.413147,49.778206,2.777344,2454.0,1626.0
1,29.695017,93.2625,-1.611572,-1.919434,0.100588,0.124390,-5.054688,-2.496094,514.0,-5.484375,390.0,-1.850586,20.265625,-1.706299,52.056957,6.945312,1174.0,1016.0


## Some charge flip work

In [18]:
!wget http://uaf-8.t2.ucsd.edu/~ewallace/chargeflipfull2016.pkl.gz

--2021-05-18 11:39:22--  http://uaf-8.t2.ucsd.edu/~ewallace/chargeflipfull2016.pkl.gz
Resolving uaf-8.t2.ucsd.edu (uaf-8.t2.ucsd.edu)... 169.228.130.7
Connecting to uaf-8.t2.ucsd.edu (uaf-8.t2.ucsd.edu)|169.228.130.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 951 [application/x-gzip]
Saving to: ‘chargeflipfull2016.pkl.gz’

100%[======================================>] 951         --.-K/s   in 0s      

2021-05-18 11:39:22 (65.2 MB/s) - ‘chargeflipfull2016.pkl.gz’ saved [951/951]



In [19]:
from Tools.helpers import yahist_2D_lookup
import gzip
import pickle
 
class charge_flip:
    def __init__(self, path):
        self.path = path
        with gzip.open(self.path) as fin:
            self.ratio= pickle.load(fin)
    
    def flip_ratio(self, lepton1, lepton2):
        """takes a dilepton event and weights it based on the 
        odds that one of the leptons has a charge flip"""

        flip1 = yahist_2D_lookup(self.ratio, lepton1.pt, abs(lepton1.eta))
        flip2 = yahist_2D_lookup(self.ratio, lepton2.pt, abs(lepton2.eta))

        flip_rate1 = (ak.prod(flip1, axis = 1) * ak.prod(1/(1-flip1), axis = 1) * ak.prod(1-flip2/(1-flip2), axis = 1)) + (ak.prod(flip2, axis = 1) * ak.prod(1/(1-flip2), axis = 1) * ak.prod(1-flip1/(1-flip1), axis = 1))

        return flip_rate1
    
    def flip_weight(self, electron):

        f_1 = yahist_2D_lookup(self.ratio, electron.pt[:,0:1], abs(electron.eta[:,0:1]))
        f_2 = yahist_2D_lookup(self.ratio, electron.pt[:,1:2], abs(electron.eta[:,1:2]))

        # I'm using ak.prod and ak.sum to replace empty arrays by 1 and 0, respectively
        weight = ak.sum(f_1/(1-f_1), axis=1)*ak.prod(1-f_2/(1-f_2), axis=1) + ak.sum(f_2/(1-f_2), axis=1)*ak.prod(1-f_1/(1-f_1), axis=1)

        return weight

In [20]:
cf = charge_flip('chargeflipfull2016.pkl.gz')

ModuleNotFoundError: No module named 'yahist'

In [12]:
electrons.matched_gen.parent.pdgId

<Array [[], [24], [], ... [], [], [-24]] type='5000 * var * ?int32[parameters={"...'>

In [ ]:
gen_matched_electrons = electrons[( (electrons.genPartIdx >= 0) & (abs(electrons.matched_gen.pdgId)==11) )]

In [48]:
gen_matched_electrons.eta.tolist()

[[],
 [],
 [0.595947265625],
 [],
 [1.171875],
 [],
 [],
 [],
 [],
 [0.9163818359375],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [-0.32049560546875, 0.25970458984375],
 [],
 [],
 [],
 [],
 [-0.37841796875, -1.37646484375, 0.7679443359375],
 [],
 [-0.145843505859375, 0.183868408203125],
 [],
 [],
 [],
 [],
 [],
 [],
 [0.01784515380859375],
 [],
 [],
 [-0.8026123046875],
 [],
 [],
 [],
 [],
 [],
 [1.339599609375],
 [],
 [1.3369140625],
 [],
 [0.64794921875],
 [],
 [],
 [],
 [],
 [-0.7940673828125, -0.3917236328125],
 [],
 [],
 [0.33685302734375, -0.300537109375],
 [],
 [],
 [-0.03604888916015625],
 [],
 [],
 [],
 [-0.843017578125],
 [],
 [],
 [],
 [0.8629150390625],
 [],
 [],
 [-1.021484375],
 [],
 [],
 [0.24481201171875],
 [0.0242156982421875],
 [0.8260498046875],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [-1.035888671875, -0.665283203125],
 [0.5714111328125],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [16]:
is_flipped =( (gen_matched_electrons.matched_gen.pdgId*(-1) == gen_matched_electrons.pdgId) & (abs(gen_matched_electrons.pdgId) == 11) )

In [49]:
flipped_electrons = gen_matched_electrons[is_flipped]

In [50]:
flipped_electrons = flipped_electrons[(ak.fill_none(flipped_electrons.pt, 0)>0)]

In [52]:
flipped_electrons.pt.tolist()

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
